# Binary Discrete Choice Models
For an individual, both observable and unobservable features can influence decisions. Utility functions for individuals which consider both observable and unobservable factors take the form:

\begin{equation}
U_{ij} = V_{ij} + \epsilon_{ij}
\end{equation}

The term $U_{ij}$ is the utility of alternative $j$ for individual $i$, $V_{ij}$ is the _deterministic_ component of the utility, 
i.e., the utility associated with the observable features, and $\epsilon_{ij}$ is the random component of the utility (error model). 

## Logit choice model
One of the most common choice models is the `logit` model. Assume the random component of the utility $U_{ij}$ is _independently and identically distributed_ (IID) across $J$ alternatives, and is [Gumbel distributed](https://en.wikipedia.org/wiki/Gumbel_distribution), 
then the probability that individual $i$ chooses alternative $j$ is given by the [logit choice model](https://en.wikipedia.org/wiki/Discrete_choice):

\begin{equation}
P_{ij} = \frac{e^{V_{ij}/\mu}}{\displaystyle \sum_{k=1}^{J}e^{V_{ik}/\mu}}\qquad{j=1,\dotsc,J}
\end{equation}

where $P_{ij}$ is the probability that individual $i$ chooses alternative $j$, $V_{ij}$ is the deterministic component of the utility, and $\mu$ is a scale parameter.

## Learning Objectives
In this example, we'll use the `logit` model to compute the probability of a `binary` choice, namely, should we by a [Tesla Model S](https://www.tesla.com/models), which we call outcome `1`, or a [Honda Odyssey](https://automobiles.honda.com/odyssey) which is outcome `0`. In particular, we will:

* Familiarize students with binary discrete choice models and Monte-Carlo simulations
* Familiarize students with [Bernoulli random variables](https://en.wikipedia.org/wiki/Bernoulli_distribution) and the simulation of binary choices

## Setup

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`


## Data
The dataset that we explore is the Tesla versus Odyssey survey presented in class. We load this dataset into the notebook using the `HondaTeslaDataSet()` function. This function stored the data in the `dataset` variable which is a [DataFrame type](https://dataframes.juliadata.org/stable/).

In [2]:
dataset = HondaTeslaDataSet()

Row,feature,exponent,Tesla,Honda
,String15,Float64,Float64,Float64
1,sustainability,0.2,5.0,3.0
2,affordability,0.1,2.0,4.0
3,styling,0.05,5.0,2.0
4,usefulness,0.3,2.0,5.0
5,costownership,0.1,4.0,2.0
6,performance,0.05,5.0,1.0
7,safety,0.2,5.0,5.0


### Analytical Logit Choice Model
The deterministic component of the random utility function $V_{ij}$ can be any of the utility functions we studied in class or one of your creations. Let's use the `log-transformed` [Cobb-Douglas utility function](https://varnerlab.github.io/CHEME-5760-Decisions-Book/unit-1-simpledecisions/utilityfunctions.html#cobb-douglas-utility-functions):

$$
\begin{equation}
\ln U = \sum_{i=1}^{m}\alpha_{i}\ln x_{i}
\end{equation}
$$

where we constrain the sum of the exponents $\alpha_{i}$ to be unity, i.e.., $\sum\alpha_{i} = 1$.

#### Implementation
We construct an instance of the `VLLogTransformedCobbDouglasUtilityFunction` model type (which holds the `Cobb-Douglas` model parameters) using the `build(...)` function, where we pass the $\alpha$-vector and base `b` as parameters:

In [13]:
model = build(VLLogTransformedCobbDouglasUtilityFunction, (
        α = dataset[:,:exponent], b = ℯ)
);

Now that we have the `VLLogTransformedCobbDouglasUtilityFunction` model, we can compute the values of the utility function using the `model(x)` short-cut syntax method:

In [14]:
V = zeros(2);
V[1] = model(dataset[:,:Tesla]);
V[2] = model(dataset[:,:Honda]);

Finally, we can compute the probability that a decision maker will choose the `Tesla` option (which is index `1`) and store this value in the variable `p`:

In [15]:
p = exp(V[1])/(exp(V[1])+exp(V[2]))

0.4883931698990063

### Bernoulli random variable
A Bernoulli random variable $X$ models a binary outcome: either `1` or `0`, 
where `1` occurs with probability $p$ and `0` occurs with probability $1-p$. 
The probability mass function (pmf) of the Bernoulli random variable $X$ is:

\begin{equation}
p_{X}(x) = \begin{cases}
    p & \text{if } x = 1 \\
    1 - p & \text{if } x = 0
  \end{cases}
\end{equation}

where $0<p<1$ is called the Bernoulli parameter. The expectation a Bernoulli random variable equals:

\begin{equation}
\mathbb{E}\left[X\right] = p
\end{equation}

while the variance $\text{Var}(X)$ equals:

\begin{equation}
\text{Var}\left[X\right] = p(1-p)
\end{equation}

Bernoulli random variables model many binary events: coin flips (`H` or `T`), `true` or `false`, `yes` or `no`, `present` or `absent`, etc.

In choosing between the `Tesla` and the `Odyssey`, we let `Tesla` be outcome `1`, while the `Odyssey` can be outcome `0`. If this is true, the `p` computed from the `logit` model equals the `Bernoulli parameter`.

#### Implementation
We create a `Bernoulli` distribution with parameter `p` using the function `Bernoulli(p)`, which is exported by the [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) package. We store this distribution in the varaible `d`:

In [19]:
d = Bernoulli(p);

Next, we generate $N_{s}$ samples drawn from the Bernoulli distribution `d`:

In [17]:
Nₛ = 10000
S = rand(d,Nₛ);

In [18]:
idx = findall(x-> x == 1, S) |> x-> length(x)/Nₛ

0.4941

### Monte-Carlo Simulation of Discrete Binary Choice

In [9]:
ϵ = Gumbel(0,π/6)
N = 10000
D₁ = V[1] .+ rand(ϵ,N);
D₂ = V[2] .+ rand(ϵ,N);

In [10]:
D = [D₁ D₂];
choice_vector = Array{Int64,1}();
for i ∈ 1:N
    if (D[i,1] > D[i,2])
        push!(choice_vector,0)
    else
        push!(choice_vector,1)
    end
end

In [11]:
findall(x-> x == 0, choice_vector) |> x-> length(x)/N

0.5013